In [55]:
import pandas as pd
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [56]:
train=pd.read_csv("train.csv")
ytrain=train["Transported"]
xtrain=train.drop(["Transported","PassengerId","Name"],axis=1)

test=pd.read_csv("test.csv")
xtest=test.drop(["PassengerId","Name"],axis=1)

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [58]:
category_list=["HomePlanet",'Destination',"Age","VIP",'cabindeck','cabinside']
num_list=["Age",'RoomService','FoodCourt','CryoSleep','ShoppingMall','Spa','VRDeck','cabinnum']

In [59]:
def minus_mean(list1):
    print(list1)
    list1=pd.DataFrame(list1)
    mean3=list1.mean(axis=0)
    print(mean3)
    list1-=mean3
    std=list1.std(axis=0)
    list1/=std
    print(list1)
    return list1

In [60]:
def boolprocess(list2):
    list3=[]
    for i in range(list2.shape[0]):
        bool1=list2[i]
        if bool1==True:
            list3.append(1)
        elif bool1==False:
            list3.append(0)
        else:
            list3.append(0.5)
    return pd.DataFrame(list3)

In [61]:
def fill(xtrain,key):
    xtrain['isna']=0
    mask=xtrain[key].isnull()  
    xtrain.loc[mask,'isna']=1
    xtrain[key].fillna(0,inplace=True)
    return xtrain[key]

In [62]:
def input_preprocess(input):
    cabin=input["Cabin"]
    cabin1=[]
    cabin2=[]
    cabin2sum=0
    cabin2num=0
    cabin3=[]
    output=pd.DataFrame([])
    for i in range(cabin.shape[0]):
        try:
            cabin1.append(cabin[i][0])
            cabin2.append(int(cabin[i][2:-2]))
            cabin3.append(cabin[i][-1])
            cabin2sum+=int(cabin[i][2:-2])
            cabin2num+=1
        except:
            cabin1.append('F')
            cabin2.append(600)
            cabin3.append('S')
    cabin1=pd.DataFrame(cabin1,columns=["cabindeck"])
    cabin2=pd.DataFrame(cabin2,columns=["cabinnum"])
    cabin3=pd.DataFrame(cabin3,columns=["cabinside"])

    #output=pd.concat([output,cabin1,cabin2,cabin3],axis=1)
    input=pd.concat([input,cabin1,cabin2,cabin3],axis=1)

    category_list=["HomePlanet",'Destination','cabindeck','cabinside']
    num_list=["Age",'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','cabinnum']
    bool_list=["CryoSleep","VIP"]

    for i in bool_list+category_list:
        output=pd.concat([output,pd.get_dummies(input[i],prefix=i)],axis=1)

    for i in num_list:
        output=pd.concat([output,pd.DataFrame(preprocessing.scale(input[i]),columns=[i])],axis=1)
        output[i]=fill(output,i)
    output=output.drop(["isna"],axis=1)
    return output

In [63]:
'''
    for k in num_list:
        mean1=xtrain[k].mean
        print(mean1)
        list1=xtrain[k]
        print(k)
        for i in range(list1.shape[0]):
            if pd.isnull(list1[i]):
                xtrain[k][i]=mean1
        xtrain[k]=minus_mean(xtrain[k])
'''

'\n    for k in num_list:\n        mean1=xtrain[k].mean\n        print(mean1)\n        list1=xtrain[k]\n        print(k)\n        for i in range(list1.shape[0]):\n            if pd.isnull(list1[i]):\n                xtrain[k][i]=mean1\n        xtrain[k]=minus_mean(xtrain[k])\n'

In [64]:
for i in range(ytrain.shape[0]):
    if  ytrain[i]:
        ytrain[i]=1
    else:
        ytrain[i]=0

C:\Users\yutia\AppData\Local\Temp\ipykernel_45768\1517399558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ytrain[i]=0


In [65]:
xtrain=input_preprocess(xtrain)

In [66]:
xtrain.keys()

Index(['CryoSleep_False', 'CryoSleep_True', 'VIP_False', 'VIP_True',
       'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'cabindeck_A', 'cabindeck_B', 'cabindeck_C',
       'cabindeck_D', 'cabindeck_E', 'cabindeck_F', 'cabindeck_G',
       'cabindeck_T', 'cabinside_P', 'cabinside_S', 'Age', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'cabinnum'],
      dtype='object')

In [68]:
data=pd.concat([xtrain,ytrain],axis=1)
data.to_csv("after.csv",index=0)

In [70]:
input_preprocess(xtest).to_csv("testafter.csv",index=0)

In [ ]:
import seaborn as sns
sns.pairplot(pd.concat([xtrain,ytrain],axis=1),hue="Transported")